1. 使用前可能需要更新selenium对应Chrome版本的插件: https://chromedriver.storage.googleapis.com/index.html  
2. 你可以先跑一下前两个代码块，看错误提示是哪个Chrome版本  
3. 然后将解压得到的chromedriver.exe移动到chrome.exe所在的目录，一般来说（例如本例）该位置是C:\Program Files (x86)\Google\Chrome\Application  
4. （如电脑曾使用过selenium则不需此步）最后，配置环境变量，右键此电脑（注意不是快捷方式）-属性-高级系统设置-环境变量-系统环境变量-Path-编辑-新建，添加好上条的目录位置
5. 参考教程: https://blog.csdn.net/orange_xiang/article/details/82924296

In [22]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep

import pandas as pd 
import numpy as np
import math

In [23]:
driver = webdriver.Chrome()
driver.implicitly_wait(10)
url = "https://bj.ke.com/ershoufang/"
driver.get(url)

# 收集街区页面链接
若已有链接的csv文件可跳过此块

In [ ]:
# 进入地域分类
driver.find_element(By.ID, 'areaTab').click()

In [70]:
# 收集所有街区的页面地址
# 因为任一筛选有30条×100页=3000条上限，无法获取全部8万+条信息
# 故细化筛选条件到街区，以实现每个分类少于100页
# 要选取多个elements，应在母节点的(full) xpath后加/*
# 点击按钮刷新页面就会丢失引用，所以提前把子按钮的数量存储好。麻烦的问题，每次刷新后要用都要重新find一次
block_links = []
num_districts = len(driver.find_elements(By.XPATH, '/html/body/div[1]/div[3]/div[1]/dl[2]/dd/div[1]/div[1]/*'))
#print(num_districts)
i = 1
while True:  # 遍历所有行政区
    district = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[1]/dl[2]/dd/div[1]/div[1]/a[{}]'.format(i))
    district_text = district.text
    district.click()
    #要获取某一类标签，直接用标签名，如div及本例中a。限定属性则用[@attribute=""]。还可以加逻辑判断符如[... and ...]
    num_blocks = len(driver.find_elements(By.XPATH, '/html/body/div[1]/div[3]/div[1]/dl[2]/dd/div[1]/div[2]/a'))
    #print(num_blocks)
    j=1
    while True:  # 遍历一行政区下所有街区
        block = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[1]/dl[2]/dd/div[1]/div[2]/a[{}]'.format(j))
        block_text = block.text
        print(i,j,district_text,block_text)
        href = block.get_attribute('href')
        print(href)
        block_links.append(href)
        j += 1
        if j > num_blocks:
            break 
        # 网页问题：点击东城区下的地安门，会跳转到西城区下的地安门，诸如此类。故每次爬取完一个街区后，要强制点击回到原行政区
        driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[1]/dl[2]/dd/div[1]/div[1]/a[{}]'.format(i)).click()
    i += 1
    if i > num_districts:
        break

1 1 东城 安定门
https://bj.ke.com/ershoufang/andingmen/
1 2 东城 安贞
https://bj.ke.com/ershoufang/anzhen1/
1 3 东城 朝阳门内
https://bj.ke.com/ershoufang/chaoyangmennei1/
1 4 东城 朝阳门外
https://bj.ke.com/ershoufang/chaoyangmenwai1/
1 5 东城 崇文门
https://bj.ke.com/ershoufang/chongwenmen/
1 6 东城 灯市口
https://bj.ke.com/ershoufang/dengshikou/
1 7 东城 地安门
https://bj.ke.com/ershoufang/dianmen/
1 8 东城 东单
https://bj.ke.com/ershoufang/dongdan/
1 9 东城 东花市
https://bj.ke.com/ershoufang/donghuashi/
1 10 东城 东四
https://bj.ke.com/ershoufang/dongsi1/
1 11 东城 东直门
https://bj.ke.com/ershoufang/dongzhimen/
1 12 东城 工体
https://bj.ke.com/ershoufang/gongti/
1 13 东城 广渠门
https://bj.ke.com/ershoufang/guangqumen/
1 14 东城 和平里
https://bj.ke.com/ershoufang/hepingli/
1 15 东城 建国门内
https://bj.ke.com/ershoufang/jianguomennei/
1 16 东城 建国门外
https://bj.ke.com/ershoufang/jianguomenwai/
1 17 东城 交道口
https://bj.ke.com/ershoufang/jiaodaokou/
1 18 东城 金宝街
https://bj.ke.com/ershoufang/jinbaojie/
1 19 东城 六铺炕
https://bj.ke.com/ershoufang/liupukang/
1 20 东

4 39 海淀 圆明园
https://bj.ke.com/ershoufang/yuanmingyuan/
4 40 海淀 玉泉路
https://bj.ke.com/ershoufang/yuquanlu11/
4 41 海淀 皂君庙
https://bj.ke.com/ershoufang/zaojunmiao/
4 42 海淀 知春路
https://bj.ke.com/ershoufang/zhichunlu/
4 43 海淀 中关村
https://bj.ke.com/ershoufang/zhongguancun/
4 44 海淀 紫竹桥
https://bj.ke.com/ershoufang/zizhuqiao/
5 1 丰台 北大地
https://bj.ke.com/ershoufang/beidadi/
5 2 丰台 北京南站
https://bj.ke.com/ershoufang/beijingnanzhan1/
5 3 丰台 菜户营
https://bj.ke.com/ershoufang/caihuying/
5 4 丰台 草桥
https://bj.ke.com/ershoufang/caoqiao/
5 5 丰台 成寿寺
https://bj.ke.com/ershoufang/chengshousi1/
5 6 丰台 大红门
https://bj.ke.com/ershoufang/dahongmen/
5 7 丰台 方庄
https://bj.ke.com/ershoufang/fangzhuang1/
5 8 丰台 丰台其它
https://bj.ke.com/ershoufang/fengtaiqita1/
5 9 丰台 广安门
https://bj.ke.com/ershoufang/guanganmen/
5 10 丰台 和义
https://bj.ke.com/ershoufang/heyi/
5 11 丰台 花乡
https://bj.ke.com/ershoufang/huaxiang/
5 12 丰台 角门
https://bj.ke.com/ershoufang/jiaomen/
5 13 丰台 旧宫
https://bj.ke.com/ershoufang/jiugong1/
5 14 丰台 看丹桥
htt

16 11 密云 巨各庄镇
https://bj.ke.com/ershoufang/jugezhuangzhen/
16 12 密云 密云其它
https://bj.ke.com/ershoufang/miyunqita11/
16 13 密云 密云镇
https://bj.ke.com/ershoufang/miyunzhen/
16 14 密云 穆家峪镇
https://bj.ke.com/ershoufang/mujiayuzhen/
16 15 密云 石城镇
https://bj.ke.com/ershoufang/shichengzhen/
16 16 密云 十里堡镇
https://bj.ke.com/ershoufang/shilipuzhen/
16 17 密云 太师屯镇
https://bj.ke.com/ershoufang/taishitunzhen/
16 18 密云 檀营
https://bj.ke.com/ershoufang/tanying/
16 19 密云 新城子镇
https://bj.ke.com/ershoufang/xinchengzizhen/
16 20 密云 西田各庄镇
https://bj.ke.com/ershoufang/xitiangezhuangzhen/
16 21 密云 溪翁庄镇
https://bj.ke.com/ershoufang/xiwengzhuangzhen/
17 1 延庆 延庆其它
https://bj.ke.com/ershoufang/yanqingqita1/


In [ ]:
block_links = block_links.drop_duplicates()

In [90]:
len(block_links)

322

In [79]:
pd.Series(block_links).rename('链接').to_csv('贝壳北京二手房街区筛选页面链接.csv')

# 逐街区爬取房源条目

In [19]:
links = pd.read_csv('贝壳二手房分街区筛选页面链接.csv',index_col=[0]).loc[:,'链接'].drop_duplicates()
links

0               https://bj.ke.com/ershoufang/andingmen/
1                 https://bj.ke.com/ershoufang/anzhen1/
2         https://bj.ke.com/ershoufang/chaoyangmennei1/
3         https://bj.ke.com/ershoufang/chaoyangmenwai1/
4             https://bj.ke.com/ershoufang/chongwenmen/
                             ...                       
317               https://bj.ke.com/ershoufang/tanying/
318        https://bj.ke.com/ershoufang/xinchengzizhen/
319    https://bj.ke.com/ershoufang/xitiangezhuangzhen/
320      https://bj.ke.com/ershoufang/xiwengzhuangzhen/
321          https://bj.ke.com/ershoufang/yanqingqita1/
Name: 链接, Length: 265, dtype: object

In [20]:
links = links.to_list()

In [38]:
items = []
for link in links:
    driver.get(link)
    
    # 记录页数。页码部件变化多端，故采用总数/30的方法。因为贝壳每页最多显示30个条目
    total_entry = driver.find_element(By.XPATH,'/html/body/div[1]/div[4]/div[1]/div[2]/div[1]/h2/span').text
    total_entry = int(total_entry)
    num_pages = math.ceil(total_entry / 30)
    print('{} pages, {} entries for {}'.format(num_pages, total_entry, link))
    i = 1
    
    # 爬取
    while i <= num_pages:
        content = driver.find_elements(By.XPATH,'/html/body/div[1]/div[4]/div[1]/div[4]/ul')
        
        # 解决爬取进行时偶然房源条目数量变动的问题
        if len(content) == 0:
            break
        content = content[0]
        
        #print(len(content.find_elements(By.CLASS_NAME,'info.clear')))
        for entry in content.find_elements(By.CLASS_NAME,'info.clear'):
            item = {}
            item['标题'] = entry.find_element(By.XPATH,'./div[1]/a').text  # 相对路径以./开头
            item['网页'] = entry.find_element(By.XPATH,'./div[1]/a').get_attribute('href')
            item['区域'] = entry.find_element(By.XPATH,'./div[2]/div[1]/div/a').text
            item['详情'] = entry.find_element(By.XPATH,'./div[2]/div[2]').text
            item['关注'] = entry.find_element(By.XPATH,'./div[2]/div[3]').text
            
            tags = []
            for span in entry.find_elements(By.XPATH,'./div[2]/div[4]/span'):
                tag = span.text
                tags.append(tag)
            item['标签'] = tags
            
            item['总价'] = entry.find_element(By.XPATH,'./div[2]/div[5]/div[1]/span').text + \
                            entry.find_element(By.XPATH,'./div[2]/div[5]/div[1]/i[2]').text
            item['均价'] = entry.find_element(By.XPATH,'./div[2]/div[5]/div[2]').text
            items.append(item)
            #print('已采集{}个房源信息'.format(len(items)))
        
        print('已采集{}条房源信息'.format(len(items)))
        i += 1
        new_link = link + 'pg{}'.format(i)
        print('turning to page {}'.format(new_link))
        driver.get(new_link)
        
    print()       

8 pages, 191 entries for https://bj.ke.com/ershoufang/andingmen/
已采集30条房源信息
turning to page https://bj.ke.com/ershoufang/andingmen/pg2
已采集60条房源信息
turning to page https://bj.ke.com/ershoufang/andingmen/pg3
已采集90条房源信息
turning to page https://bj.ke.com/ershoufang/andingmen/pg4
已采集120条房源信息
turning to page https://bj.ke.com/ershoufang/andingmen/pg5
已采集150条房源信息
turning to page https://bj.ke.com/ershoufang/andingmen/pg6
已采集180条房源信息
turning to page https://bj.ke.com/ershoufang/andingmen/pg7
已采集191条房源信息
turning to page https://bj.ke.com/ershoufang/andingmen/pg8

15 pages, 413 entries for https://bj.ke.com/ershoufang/anzhen1/
已采集221条房源信息
turning to page https://bj.ke.com/ershoufang/anzhen1/pg2
已采集251条房源信息
turning to page https://bj.ke.com/ershoufang/anzhen1/pg3
已采集281条房源信息
turning to page https://bj.ke.com/ershoufang/anzhen1/pg4


KeyboardInterrupt: 

In [96]:
df = pd.DataFrame(items)
df

,标题,网页,区域,详情,关注,标签,总价,均价
0,工人日报社的家属院，管理好；南北通透，生活便利,https://bj.ke.com/ershoufang/101114510314.html,安德路乙61号院,中楼层 (共12层) 3室1厅 | 66.65平米 | 南 北,11人关注 / 2天前发布,"[近地铁, 满五年, 新上]",888万,"133,234元/平"
1,东城，安定门，安外大街3号院，带电梯两居室,https://bj.ke.com/ershoufang/101114470063.html,安外大街3号院,低楼层 (共12层) 2室1厅 | 51.91平米 | 东,15人关注 / 5天前发布,"[满五年, VR看装修, 随时看房, 新上]",590万,"113,659元/平"
2,青年湖公园旁边两居室，业主诚心出售。,https://bj.ke.com/ershoufang/101114453139.html,安德路55号院,顶层 (共3层) | 1995年建 | 2室0厅 | 47.07平米 | 东 北,9人关注 / 7天前发布,"[近地铁, 满两年, VR看装修, 新上]",540万,"114,723元/平"
3,安外东河沿半地下2米高大窗户全在地上20万精装大两居,https://bj.ke.com/ershoufang/101114451333.html,安外东河沿,地下室 (共6层) | 1998年建 | 2室1厅 | 71.7平米 | 南 北,16人关注 / 7天前发布,"[近地铁, 满五年, VR看装修, 新上]",635万,"88,564元/平"
4,安交新上03年大两居 方家胡同 永康胡同1号院 满五唯一,https://bj.ke.com/ershoufang/101114430629.html,永康胡同1号院,中楼层 (共7层) 2室1厅 | 67.36平米 | 东 西,21人关注 / 8天前发布,"[近地铁, 满五年, VR看装修, 新上]",980万,"145,487元/平"
...,...,...,...,...,...,...,...,...
21841,紫东苑 满五年唯一住房 精装修,https://bj.ke.com/ershoufang/101114191048.html,紫东苑,低楼层 (共22层) 3室1厅 | 126.99平米 | 西,24人关注 / 30天前发布,"[近地铁, 满五年, VR房源]",725万,"57,092元/平"
21842,劲松北工大商圈，南北两居，大客厅，93年房龄,https://bj.ke.com/ershoufang/101114185731.html,农光南路,顶层 (共6层) | 1993年建 | 2室1厅 | 64.95平米 | 南 北,22人关注 / 30天前发布,"[近地铁, VR房源]",385万,"59,277元/平"
21843,劲松二区 板楼三居 格局方正 南北通透 智慧型社区,https://bj.ke.com/ershoufang/101114176058.html,劲松二区,顶层 (共6层) | 1980年建 | 3室1厅 | 72.66平米 | 南 北,37人关注 / 1月前发布,"[近地铁, 满两年, VR房源]",415万,"57,116元/平"
21844,二环边 电梯直达 采光好 诚心出售,https://bj.ke.com/ershoufang/101114162111.html,劲松八区,高楼层 (共12层) | 1984年建 | 1室0厅 | 43.69平米 | 东,8人关注 / 1月前发布,"[满两年, VR房源]",278万,"63,631元/平"


In [97]:
df.to_csv('贝壳北京二手房房源条目.csv')

# draft

In [68]:
# 要选取多个elements，应在母节点的(full) xpath后加/*
# 点击按钮刷新页面就会丢失引用，所以提前把子按钮的数量存储好。麻烦的问题，每次刷新后要用都要重新find一次
num_districts = len(driver.find_elements(By.XPATH, '/html/body/div[1]/div[3]/div[1]/dl[2]/dd/div[1]/div[1]/*'))
print(num_districts)
i = 1
items = []
while True:  # 遍历所有行政区
    district = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[1]/dl[2]/dd/div[1]/div[1]/a[{}]'.format(i))
    district_text = district.text
    district.click()
    #要获取某一类标签，直接用标签名，如div及本例中a。限定属性则用[@attribute=""]。还可以加逻辑判断符如[... and ...]
    num_blocks = len(driver.find_elements(By.XPATH, '/html/body/div[1]/div[3]/div[1]/dl[2]/dd/div[1]/div[2]/a'))
    print(num_blocks)
    j=1
    while True:  # 遍历一行政区下所有街区
        block = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[1]/dl[2]/dd/div[1]/div[2]/a[{}]'.format(j))
        block_text = block.text
        print(i,j,district_text,block_text)
        block.click()
        sleep(1)
        
        # 爬取
        content = driver.find_element(By.XPATH,'/html/body/div[1]/div[4]/div[1]/div[4]/ul')
        #print(len(content.find_elements(By.CLASS_NAME,'info.clear')))
        for entry in content.find_elements(By.CLASS_NAME,'info.clear'):
            item = {}
            item['标题'] = entry.find_element(By.XPATH,'./div[1]/a').text  # 相对路径以./开头
            item['网页'] = entry.find_element(By.XPATH,'./div[1]/a').get_attribute('href')
            item['区域'] = entry.find_element(By.XPATH,'./div[2]/div[1]/div/a').text
            item['详情'] = entry.find_element(By.XPATH,'./div[2]/div[2]').text
            item['关注'] = entry.find_element(By.XPATH,'./div[2]/div[3]').text
            
            tags = []
            for span in entry.find_elements(By.XPATH,'./div[2]/div[4]/span'):
                tag = span.text
                tags.append(tag)
            item['标签'] = tags
            
            item['总价'] = entry.find_element(By.XPATH,'./div[2]/div[5]/div[1]/span').text + \
                            entry.find_element(By.XPATH,'./div[2]/div[5]/div[1]/i[2]').text
            item['均价'] = entry.find_element(By.XPATH,'./div[2]/div[5]/div[2]').text
            print(item)
            items.append(item)
            print('已采集{}个房源信息'.format(len(items)))
        
        # 翻页
        
        
        j += 1
        if j > num_blocks:
            break 
        # 网页问题：点击东城区下的地安门，会跳转到西城区下的地安门，诸如此类。故每次爬取完一个街区后，强制点击回到原行政区
        driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[1]/dl[2]/dd/div[1]/div[1]/a[{}]'.format(i)).click()
    i += 1
    if i > num_districts:
        break

17
28
1 1 东城 安定门


KeyboardInterrupt: 

In [26]:
content = driver.find_elements(By.XPATH,'/html/body/div[1]/div[4]/div[1]/div[4]/ul')

In [33]:
len(content)

0